In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir
from k12libs.utils.nb_easy import k12ai_get_top_dir
from k12libs.utils.nb_widget import k12ai_schema_parse

import os
import json
import _jsonnet

## Layout Type Support

- [x] **page** 根节点
- [x] **tab** 页标签节点, 一般每个tab里面的内容最好是独立的一个模块
- [x] **accordion** 折叠卡节点, 一般用在tab节点中, 作为单一模块的子项设置
- [x] **navigation** 导航卡节点, 和accordion相似, 展示形态不太一样, 只为了页面美观, 非必须
- [x] **object** 对象节点, 当前布局不变, 将它的子节点布局到当前布局中, 比如当前布局是水平的, 则子节点会水平放置
- [x] **H** 水平布局节点, 在当前布局中创建一个水平布局, 其子节点将会水平布局
- [x] **V** 垂直布局节点, 在当前布局中创建一个垂直布局, 其子节点将会垂直布局
- [x] **~~HV~~** 没有存在的必要,废除  
- [ ] **~~\_ignore\_~~** 或者无type, 同object, 不同之处在于"object"节点里如果存在"name"字段会右侧黑体展示分割标题.


## Layout Note Attribute

因为`object, H, V`功能类似, 只是为了改变布局, 并没有真正的控件样式, 所以归为一类, 叫"简单布局", `_ignore_`与`object`节点不同主要在于如果`object`节点中`name`字段, 则在右侧黑体显示分割标题.

### 1. 简单布局

```json
{
    "_id_": "a.b.c", // [O] 字符串, 没有实际意义
    "type": "",      // [O] 字符串, 节点的类型, object, _ignore_, H, V, 如果没有type字段默认为'object'
    "name": {},      // [O] 对象, 形如: {en:"test", cn:"测试"}, 若存在, 右侧粗体显示该名字,作为分割标题
    "objs": [        // [M] 该布局下的子节点
    ]
}
```

### 2. page节点

```json
{
    "_id_": "a.b.c", // [O] 字符串, 没有实际意义
    "type": "page",  // [M] 字符串, 节点的类型
    "name": {},      // [O] 对象, 形如: {en:"test", cn:"测试"}, 没有实际意义
    "objs": [        // [M] 该布局下的子节点
    ]
}
```

### 3. tab节点

```json
{
    "_id_": "a.b.c", // [O] 字符串, 没有实际意义
    "type": "tab",   // [M] 字符串, 节点的类型
    "name": {},      // [M] 对象, 形如: {en:"test", cn:"测试"}, 每个标签页显示的名字
    "objs": [        // [M] 该布局下的子节点
        {
            "name": {}           // [M] 对象, 标签卡子项的名字, 定义同上
            "type": '_ignore_',  // [O] 标签卡子项节点类型, 可以不含此字段
            "objs": [            // [M] 标签卡子项节点内容
            ]
        },
        {
            "name": {}
            "objs": [
            ]
        },
    ]
}
```

### 4. accordion节点

```json
{
    "_id_": "a.b.c",      // [O] 字符串, 没有实际意义
    "type": "accordion",  // [M] 字符串, 节点的类型
    "name": {},           // [M] 对象, 形如: {en:"test", cn:"测试"}, 每个折叠卡显示的名字
    "objs": [             // [M] 该布局下的子节点
        {
            "name": {}           // [M] 对象, 折叠卡子项的名字, 定义同上
            "type": '_ignore_',  // [O] 折叠卡子项节点类型, 可以不含此字段
            "objs": [            // [M] 折叠卡子项节点内容
            ]
        },
        {
            "name": {}
            "objs": [
            ]
        },
    ]
}
```

### 5. navigation节点

```json
{
    "_id_": "a.b.c",        // [O] 字符串, 没有实际意义
    "type": "navigation",   // [M] 字符串, 节点的类型
    "name": {},             // [O] 对象, 形如: {en:"test", cn:"测试"}, 若存在可以显示名字
    "objs": [               // [M] 该布局下的子节点, 共有len(objs)个导航子项按钮
        {
            "name": {}           // [M] 对象, 导航卡子项的名字, 定义同上
            "type": '_ignore_',  // [O] 字符串, 导航卡子项节点类型, 可以不含此字段
            "objs": [            // [M] 对象, 导航卡子项节点内容
            ]
        },
        {
            "name": {} 
            "objs": [
            ]
        },
    ]
}
```

<div class="alert alert-info"><p>
    注意: 如果层级很多, 子节点的排版显示很紧密, 所以如果简单布局节点(只针对object, H, V)含有name属性, 这表示要在该布局的右侧粗体字体显示该name, 作为分割标题.
</p></div>

In [3]:
schema_dir = os.path.join(k12ai_get_top_dir(), 'k12libs', 'templates', 'schema')
!ls $schema_dir 

basic.jsonnet	   k12ai_all_type.jsonnet    k12ai_complex_type.jsonnet  old
default.libsonnet  k12ai_basic_type.jsonnet  k12ai_layout_type.jsonnet


In [4]:
# basic schema template file
basic_file = os.path.join(schema_dir, 'k12ai_layout_type.jsonnet')
!cat $basic_file

// @file basic.jsonnet
// @brief
// @author QRS
// @version 1.0
// @date 2019-12-18 21:08

local default_schema = import 'default.libsonnet';

{
    description: |||
        K12 Data Template Model For Layout
    |||,

    type: 'page',
    objs: [
        {
            type: 'tab',
            objs: [
                {
                    name: { en: 'tab1', cn: '页标签-1' },
                    objs: [
                        {
                            type: 'accordion',
                            objs: [
                                {
                                    name: { en: 'accordion1', cn: '折叠卡-1' },
                                    objs: [
                                        {
                                            name: { en: 'navigation1', cn: '导航栏-1' },
                                            type: 'navigation',
                                            objs: [
                                                {
                                     

In [5]:
basic_json = _jsonnet.evaluate_file(basic_file)
k12ai_print(basic_json, indent=2)

{
  "description": "K12 Data Template Model For Layout\n",
  "objs": [
    {
      "objs": [
        {
          "name": {
            "cn": "页标签-1",
            "en": "tab1"
          },
          "objs": [
            {
              "objs": [
                {
                  "name": {
                    "cn": "折叠卡-1",
                    "en": "accordion1"
                  },
                  "objs": [
                    {
                      "name": {
                        "cn": "导航栏-1",
                        "en": "navigation1"
                      },
                      "objs": [
                        {
                          "name": {
                            "cn": "选项卡-1",
                            "en": "group1"
                          },
                          "objs": [
                            {
                              "_id_": "_js.text.a.b.k12text",
                              "default": "我的右上角\n\n**有**\n\n粗体黑字的分割标题\n",
            

In [6]:
k12ai_schema_parse(json.loads(basic_json), lan='cn', debug=False);

Box(children=(VBox(children=(Tab(children=(VBox(children=(Accordion(children=(VBox(children=(VBox(children=(To…